In [2]:
import matplotlib as plt 
import numpy as np 
from matplotlib import colors
from PIL import Image, ImageDraw
import random
import pandas as pd
from collections import defaultdict
import random

In [3]:
def createList(r1, r2):
        if (r1 == r2):
            return r1
        else:
            res = []
            while(r1 != r2):
                res.append(r1)
                r1 += 1
                if (r1 > 50):
                    r1 = r1-50
            res.append(r2)
        return res 

In [4]:
class Graph():
    
    def __init__(self):
        
        #create graph
        self.graph = {}
        for i in range(1,51):
            if i-1 == 0:
                self.graph[i] = [2, 50]
            elif i+1 == 51:
                self.graph[i] = [1, 49]
            else:
                self.graph[i] = [i+1, i-1]
        
        #create list of nodes
        self.nodes = createList(1, 50)
                
    def addEdge(self):
        boo1 = True
        while boo1:
            if len(self.nodes) == 0:
                boo1 = False
                break
            val = random.choice(self.nodes)
            self.nodes.remove(val)
            valstart = val-5
            valend = val+5
            if (valstart < 1):
                valstart = valstart+50
            if (valend > 50):
                valend = valend-50
            val_nodes = createList(valstart, valend)
            if(val == 1):
                val_nodes.remove(50)
            elif(val == 50):
                val_nodes.remove(1)
            else:
                val_nodes.remove(val-1)
                val_nodes.remove(val+1)
            boo2 = False
            while len(val_nodes) > 0:
                newnode = random.choice(val_nodes)
                if newnode in self.nodes:
                    self.nodes.remove(newnode)
                    self.graph[val].append(newnode)
                    self.graph[newnode].append(val)
                    boo2 = True
                    break
                else:
                    val_nodes.remove(newnode)
                    
            if boo2:
                break
        if boo1 == False:
            return False
        else:
            return True

In [5]:
#start position, end position, backwards path to take
#returns path
def BFS(start, end, graph):
 
    # a queue of the vertices whose path is to be scanned
    # boolean array for checking if node has been visited at least once or not
    # all moves in path set to -1
    queue = []
    visited = [False for i in range(51)]
    back_path = [-1 for i in range(51)]
    
    # start is visited and added to the queue
    visited[start] = True
    queue.append(start)

    # BFS algorithm - look at all neighbors first
    while (len(queue) != 0):
        u = queue[0]
        queue.pop(0)
        for i in range(len(graph[u])):
            if (visited[graph[u][i]] == False):
                visited[graph[u][i]] = True
                back_path[graph[u][i]] = u
                queue.append(graph[u][i])

                # We stop BFS when we find
                # destination.
                if (graph[u][i] == end):
                    path = []
                    crawl = end
                    path.append(crawl)
                    
                    #reversing back_path and removing -1 from nodes that are not in the path to get full path
                    while (back_path[crawl] != -1):
                        path.insert(0, back_path[crawl])
                        crawl = back_path[crawl]
                    return path

    return [end]

In [6]:
class Predator():
    
    def __init__(self, agentloc):
        self.pos = random.randint(1,50)
        #making sure that predator doesn't start where the agent is
        while self.pos == agentloc:
            self.pos = random.randint(1,50)
        
    def pred_move(self, g, agentpos):
        
        #getting all next possible moves of predator
        neighbors = g.graph[self.pos]
        path = []
        shortest_path = []
        path_length = 50
        #finding shortest path to agent using BFS
        for neighbor in neighbors:
            path = BFS(neighbor, agentpos, g.graph)
            
            #saving shortest path(s)
            if(len(path) <= path_length):
                if(len(path) == path_length):
                    shortest_path.append(path)
                else:
                    shortest_path = []
                    shortest_path.append(path)
                    path_length = len(path)
        
        #randomly picking path of the shortest paths and moving to next neighbor
        new_path = random.randint(0,(len(shortest_path)-1))
        self.pos = shortest_path[new_path][0]
        return self

    def pred_distracted_move(self, g, agentpos): #predator moving with probability 0.6 to node closer to agent and probability 0.4 to any other random neighbour
        
        #getting all next possible moves of predator
        neighbors = g.graph[self.pos]
        path = []
        shortest_path = []
        path_length = 50
        #finding shortest path to agent using BFS
        for neighbor in neighbors:
            path = BFS(neighbor, agentpos, g.graph)
            
            #saving shortest path(s)
            if(len(path) <= path_length):
                if(len(path) == path_length):
                    shortest_path.append(path)
                else:
                    shortest_path = []
                    shortest_path.append(path)
                    path_length = len(path)
        
        #randomly picking path of the shortest paths and moving to next neighbor
        new_path = random.randint(0,(len(shortest_path)-1))
        chance = random.randint(1,100)
        if chance <= 60:
            self.pos = shortest_path[new_path][0]
        else:
            self.pos = random.choice(neighbors)
        return self

In [7]:
class Prey():
    
    def __init__(self, agentloc):
        self.pos = random.randint(1,50)
        #making sure prey doesn't start where agent is
        while self.pos == agentloc:
            self.pos = random.randint(1,50)
        
    def prey_move(self, g):
        prey_loc_neighbours = [self.pos]
        for i in g.graph[self.pos]:
            prey_loc_neighbours.append(i)
        #randomly choose neighbor to move to (or staying put)
        self.pos = random.choice(prey_loc_neighbours)
        return self

In [8]:
def Agent7(g):
    agent_pos = random.randint(1,50)
    predator_7 = Predator(agent_pos)
    prey_7 = Prey(agent_pos)
    fail = 0
    steps = 0
    path = []
    path.append(agent_pos)
    
    belief_vector_pred =  [(1/50)] * 50    #initializing belief-states
    #we know position of predator at first timestep
    for i in range(len(belief_vector_pred)): #making probability of predator being in all other nodes 0
        if i == predator_7.pos-1:
            belief_vector_pred[i] = 1
        else:
            belief_vector_pred[i] = 0 

    belief_vector_prey =  [(1/50)] * 50    #initializing belief-states
    #we know position of agent at first timestep
    for i in range(len(belief_vector_prey)): #making probability of predator being in all other nodes 0
        prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
        belief_vector_prey[agent_pos-1] = 0
        for i in range(len(belief_vector_prey)):
                belief_vector_prey[i] = belief_vector_prey[i]/prey_not

    while(fail == 0 and steps <= 1000):
        
        max_prob_pred = max(belief_vector_pred)
        list_of_positions_max_prob_pred = []
        for itr in range(len(belief_vector_pred)): #making a list of positions which have the highest probability of having the predator
            if(belief_vector_pred[itr] == max_prob_pred):
                list_of_positions_max_prob_pred.append(itr+1)
        
        #selecting closest position with equal maximum probability of having predator
        pat_len = 50
        best_position = []
        for position in list_of_positions_max_prob_pred:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_position.append(position)
                else:
                    best_position = []
                    best_position.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        possible_pred_pos = random.choice(best_position) #randomly selecting one of those positions and surveying it   

        max_prob_prey = max(belief_vector_prey)
        #print(max_prob)
        #getting max probability value in belief vector
        list_of_positions_max_prob_prey = []
        for itr in range(len(belief_vector_prey)): #making a list of positions which have the highest probability of having the prey
            if(belief_vector_prey[itr] == max_prob_prey):
                list_of_positions_max_prob_prey.append(itr+1)
        possible_prey_pos = random.choice(list_of_positions_max_prob_prey) #randomly selecting one of those positions and surveying it

        curr_to_pred = len(BFS(agent_pos, possible_pred_pos, g.graph))

        if(curr_to_pred <=3 or max_prob_pred >= 0.6): #moving if predator is too close or we have enough information about predator else surveying
            steps += 1
            #Agent 1 from here on
            neighbors = g.graph[agent_pos]
            #neighbors.append(agent_pos)
            # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and 
            # the neighbor they would move to for that path
            moves_1 = []
            moves_2 = []
            moves_3 = []
            moves_4 = []
            moves_5 = []
            moves_6 = []
            path_to_prey = []
            path_to_predator = []
            curr_to_pred = len(BFS(agent_pos, possible_pred_pos, g.graph))
            curr_to_prey = len(BFS(agent_pos, possible_prey_pos, g.graph))

                
            for neighbor in neighbors:
                
                # shortest path from neighbor to prey and shortest path from neighbor to predator
                path_to_prey = BFS(neighbor, possible_prey_pos, g.graph)
                path_to_predator = BFS(neighbor, possible_pred_pos, g.graph)
                
                #print("Prey and Predator Moves")
                #print(path_to_prey)
                #print(path_to_predator)
                if (len(path_to_prey) < curr_to_prey and len(path_to_predator) > curr_to_pred):
                    tup = (len(path_to_prey), len(path_to_predator), neighbor)
                    moves_1.append(tup)
                
                elif (len(path_to_prey) < curr_to_prey and len(path_to_predator) == curr_to_pred):
                    tup = (len(path_to_prey), len(path_to_predator), neighbor)
                    moves_2.append(tup)

                elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) > curr_to_pred):
                    tup = (len(path_to_prey), len(path_to_predator), neighbor)
                    moves_3.append(tup)

                elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) == curr_to_pred):
                    tup = (len(path_to_prey), len(path_to_predator), neighbor)
                    moves_4.append(tup)

                elif (len(path_to_predator) > curr_to_pred):
                    tup = (len(path_to_prey), len(path_to_predator), neighbor)
                    moves_5.append(tup)

                elif (len(path_to_predator) == curr_to_pred):
                    tup = (len(path_to_prey), len(path_to_predator), neighbor)
                    moves_6.append(tup)          

            
            #print()
            if(len(moves_1) > 0):
                moves_sorted = sorted(moves_1, key=lambda t: (t[0], -t[1]))
                #print(moves_sorted)
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            elif(len(moves_2) > 0):
                moves_sorted = sorted(moves_2, key=lambda t: (t[0], -t[1]))
                #print(moves_sorted)
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            elif(len(moves_3) > 0):
                moves_sorted = sorted(moves_3, key=lambda t: (t[0], -t[1]))
                #print(moves_sorted)
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            elif(len(moves_4) > 0):
                moves_sorted = sorted(moves_4, key=lambda t: (t[0], -t[1]))
                #print(moves_sorted)
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            elif(len(moves_5) > 0):
                moves_sorted = sorted(moves_5, key=lambda t: (-t[1],t[0]))
                #print(moves_sorted)
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            elif(len(moves_6) > 0):
                moves_sorted = sorted(moves_6, key=lambda t: (-t[1], t[0]))
                #print(moves_sorted)
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            if (agent_pos == predator_7.pos):
                fail = 1 #if agent has already moved to location of prey
                break
            elif (agent_pos == prey_7.pos):
                break
            else:#treat as survey and perform belief update for both
                pred_not = 1-belief_vector_pred[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                belief_vector_pred[agent_pos-1] = 0
                for i in range(len(belief_vector_pred)):
                        belief_vector_pred[i] = belief_vector_pred[i]/pred_not

                prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of prey not being in that position
                belief_vector_prey[agent_pos-1] = 0
                for i in range(len(belief_vector_prey)):
                        belief_vector_prey[i] = belief_vector_prey[i]/prey_not 

            quasi_2 = sum(belief_vector_pred)
            for i in range(len(belief_vector_pred)):
                belief_vector_pred[i]/=quasi_2 #normalization for belief of predator

        else:            
            steps += 1
            #if we arent certain of pred loc, survey node with highest probability
            if possible_pred_pos == predator_7.pos:
            #if we find pred, survey same node for prey
                for i in range(len(belief_vector_pred)):
                    if i == possible_pred_pos-1:
                        belief_vector_pred[i] = 1
                    else:
                        belief_vector_pred[i] = 0

                if possible_pred_pos == prey_7.pos:
                    for i in range(len(belief_vector_prey)):
                        if i == possible_pred_pos-1:
                            belief_vector_prey[i] = 1
                        else:
                            belief_vector_prey[i] = 0
                else:
                    prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of prey not being in that position
                    belief_vector_prey[possible_pred_pos-1] = 0
                    for i in range(len(belief_vector_prey)):
                            belief_vector_prey[i] = belief_vector_prey[i]/prey_not

            else:
                #if we dont find pred, update belief vector for pred
                pred_not = 1 - belief_vector_pred[possible_pred_pos-1]
                belief_vector_pred[possible_pred_pos-1] = 0
                for i in range(len(belief_vector_pred)):
                        belief_vector_pred[i] = belief_vector_pred[i]/pred_not

                if possible_pred_pos == prey_7.pos:
                    for i in range(len(belief_vector_prey)):
                        if i == possible_pred_pos-1:
                            belief_vector_prey[i] = 1
                        else:
                            belief_vector_prey[i] = 0
                else:
                    prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of prey not being in that position
                    belief_vector_prey[possible_pred_pos-1] = 0
                    for i in range(len(belief_vector_prey)):
                            belief_vector_prey[i] = belief_vector_prey[i]/prey_not

            quasi_1 = sum(belief_vector_pred)
            for i in range(len(belief_vector_pred)):
                belief_vector_pred[i]/=quasi_1 #normalization for predator belief, to ensure sum doesnt exceed 1


        path.append(agent_pos)

        #move prey
        prey_7.prey_move(g)

        if (prey_7.pos == agent_pos): #if prey luckily moves to location of agent
            break
        
        #move predator
        predator_7.pred_distracted_move(g, agent_pos)

        if(predator_7.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
        #belief updates for both predator and prey for next timestep
        temp_belief_vector_pred = belief_vector_pred.copy()
        for i in range(len(belief_vector_pred)):
            mag_sum=0 #no chance of predator staying at same node in next timestep
            i_neighbours = g.graph[i+1] #getting neighbours
            for bloke in i_neighbours: 
                bloke_neighbours = g.graph[bloke] #finding neighbours of neighbours one of which will be Node I
                worthy_blokes=[]
                shortest_distance = 50
                for b_n in bloke_neighbours: #finding shortest paths from each neighbour and appending shortest paths to list
                    worthy_bloke_path = BFS(b_n, agent_pos, g.graph)
                    #saving shortest path(s)
                    if(len(worthy_bloke_path) <= shortest_distance):
                        if(len(worthy_bloke_path) == shortest_distance):
                            worthy_blokes.append(b_n)
                        else:
                            worthy_blokes = []
                            worthy_blokes.append(b_n)
                            shortest_distance = len(worthy_bloke_path)
                mag_sum = mag_sum + ((temp_belief_vector_pred[bloke-1])*(0.4/len(bloke_neighbours))) #the node could be chosen on a random basis while distracted
                mag_sum = mag_sum + (0.6*(temp_belief_vector_pred[bloke-1])*worthy_blokes.count(i+1)/len(worthy_blokes)) #if the node i is the next node on one of the shortest paths to agent from neighbour, calculate 0.6 probability on the basis of number of shortest paths from that neighbour
                """neighbour_path = BFS(bloke, agent_pos, g.graph)  #finding shortest path to agent position
                #print(neighbour_path)
                if len(neighbour_path) > 1:
                    mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/len(g.graph[bloke])))
                    if (neighbour_path[1] == i+1): #if the next position after start in the shortest path is the current position
                        mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.6))
                #else:
                    #mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/(len(g.graph[bloke])-1)))"""
            belief_vector_pred[i] = mag_sum #assigning probability

        quasi_3 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_3

        temp_belief_vector_prey = belief_vector_prey.copy() #making a temporary belief vector for correct calculations
        for i in range(len(belief_vector_prey)):
            mag_sum = temp_belief_vector_prey[i]/(len(g.graph[i+1])+1)#prob of prey staying in same position
            i_neighbours = g.graph[i+1] #getting neighbours of current position
            #print(g.graph)
            #print(i+1)
            #print(i_neighbours)
            for bloke in i_neighbours:
                mag_sum = mag_sum + (temp_belief_vector_prey[bloke-1]/(len(g.graph[bloke])+1))#prob of prey moving to from neighbour bloke to node i
            belief_vector_prey[i] = mag_sum #assigning probability


    if(steps > 1000):
        fail = 2     
    return fail, path

In [12]:
wins = 0
losses = 0
timeout = 0
times = 0
win_dist = []
loss_dist = []

while times < 3000:
    g = Graph()
    graph = g.addEdge()
    while graph:
        graph = g.addEdge()
    res, path = Agent7(g)
    
    if(res == 0):
        wins += 1
        win_dist.append(len(path))
    elif(res == 1):
        losses += 1
        loss_dist.append(len(path))
    else:
        timeout += 1
    #print("Simulation Number " + str(times+1) + " completed.")
    times += 1
    
print('Number of wins: ' + str(wins))
print("Max Steps Taken During a Win: " + str(max(win_dist)-1))
print("Average Steps Taken During a Win: " + str((sum(win_dist)/wins)-1))
print('Number of losses: ' + str(losses))
print("Max Steps Taken During a Loss: " + str(max(loss_dist)-1))
print("Average Steps Taken During a Loss: " + str((sum(loss_dist)/losses)-1))
print('Number of timeouts: ' + str(timeout))
print("Success Rate: " + str((wins/times)))

Success Rate for 0.35 : 0.518
Success Rate for 0.39999999999999997 : 0.534
Success Rate for 0.44999999999999996 : 0.54
Success Rate for 0.49999999999999994 : 0.5303333333333333
Success Rate for 0.5499999999999999 : 0.5416666666666666
Success Rate for 0.6 : 0.5316666666666666
Success Rate for 0.65 : 0.5303333333333333
Success Rate for 0.7000000000000001 : 0.5446666666666666
Success Rate for 0.7500000000000001 : 0.539
Success Rate for 0.8000000000000002 : 0.5166666666666667


In [11]:
def Agent8(g):
    agent_pos = random.randint(1,50)
    predator_8 = Predator(agent_pos)
    prey_8 = Prey(agent_pos)
    fail = 0
    steps = 0
    path = []
    path.append(agent_pos)
    
    belief_vector_pred =  [(1/50)] * 50    #initializing belief-states
    #we know position of predator at first timestep
    for i in range(len(belief_vector_pred)): #making probability of predator being in all other nodes 0
        if i == predator_8.pos-1:
            belief_vector_pred[i] = 1
        else:
            belief_vector_pred[i] = 0 

    belief_vector_prey =  [(1/50)] * 50    #initializing belief-states
    #we know position of agent at first timestep
    for i in range(len(belief_vector_prey)): #making probability of predator being in all other nodes 0
        prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
        belief_vector_prey[agent_pos-1] = 0
        for i in range(len(belief_vector_prey)):
                belief_vector_prey[i] = belief_vector_prey[i]/prey_not

    while(fail == 0 and steps <= 1000):
        
        max_prob_pred = max(belief_vector_pred)
        list_of_positions_max_prob_pred = []
        for itr in range(len(belief_vector_pred)): #making a list of positions which have the highest probability of having the predator
            if(belief_vector_pred[itr] == max_prob_pred):
                list_of_positions_max_prob_pred.append(itr+1)
        possible_pred_pos = random.choice(list_of_positions_max_prob_pred) #randomly selecting one of those positions and surveying it 
        

        #selecting closest position with equal maximum probability of having predator
        pat_len = 50
        best_position = []
        for position in list_of_positions_max_prob_pred:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_position.append(position)
                else:
                    best_position = []
                    best_position.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        possible_pred_pos = random.choice(best_position) #randomly selecting one of those positions and surveying it  

        max_prob_prey = max(belief_vector_prey)
        #print(max_prob)
        #getting max probability value in belief vector
        list_of_positions_max_prob_prey = []
        for itr in range(len(belief_vector_prey)): #making a list of positions which have the highest probability of having the prey
            if(belief_vector_prey[itr] == max_prob_prey):
                list_of_positions_max_prob_prey.append(itr+1)
        possible_prey_pos = random.choice(list_of_positions_max_prob_prey) #randomly selecting one of those positions and surveying it
        
        curr_to_pred = len(BFS(agent_pos, possible_pred_pos, g.graph))

        if(curr_to_pred <= 3 or max_prob_pred >= 0.6): #moving when predator is too close or we have enough infoabout predator
            steps += 1
            neighbors = g.graph[agent_pos]
            #neighbors.append(agent_pos)
            # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and 
            # the neighbor they would move to for that path
            amazing_moves = []
            
            path_to_prey = []
            path_to_pred = []
            curr_dist_to_pred = len(BFS(agent_pos, possible_pred_pos, g.graph))
            curr_dist_to_prey = len(BFS(agent_pos, possible_prey_pos, g.graph))

            all_possible_moves = [(curr_dist_to_prey, curr_dist_to_pred, agent_pos)]

            
            for neighbour in neighbors:
                path_to_prey = BFS(neighbour, possible_prey_pos, g.graph)
                path_to_pred = BFS(neighbour, possible_pred_pos, g.graph)

                #keeping all moves handy
                tup2 = (len(path_to_prey), len(path_to_pred), neighbour)
                all_possible_moves.append(tup2)
                
                #making list of moves which only satisfy 1st condition of Agent 1
                if(len(path_to_pred) > curr_dist_to_pred and len(path_to_prey) < curr_dist_to_prey):
                    tup3 = (len(path_to_prey), len(path_to_pred), neighbour)
                    amazing_moves.append(tup3)                


            if(len(amazing_moves) > 0):             
                moves_sorted = sorted(amazing_moves, key=lambda t: (t[0], -t[1]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]): 
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]


            else: #dip into bag of all possible moves

                if (curr_dist_to_pred <= 3): #if predator too close and enough steps in the bag then run away from predator
                    moves_sorted = sorted(all_possible_moves, key=lambda t: (-t[1], t[0]))
                    move_choice=[]
                    for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                        if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                            move_choice.append(move)
                    neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                    agent_pos = neighbour_choice[2]

                else: #if step count exceeds this threshold, irrespective of predator distance make a dash to the prey
                    moves_sorted = sorted(all_possible_moves, key=lambda t: (t[0], -t[1]))
                    move_choice=[]
                    for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                        if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                            move_choice.append(move)
                    neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                    agent_pos = neighbour_choice[2]

            if (agent_pos == predator_8.pos):
                fail = 1 #if agent has already moved to location of prey
                break
            elif (agent_pos == prey_8.pos):
                break
            else:#treat as survey and perform belief update for both
                pred_not = 1-belief_vector_pred[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                belief_vector_pred[agent_pos-1] = 0
                for i in range(len(belief_vector_pred)):
                        belief_vector_pred[i] = belief_vector_pred[i]/pred_not

                prey_not = 1-belief_vector_prey[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                belief_vector_prey[agent_pos-1] = 0
                for i in range(len(belief_vector_prey)):
                        belief_vector_prey[i] = belief_vector_prey[i]/prey_not 

            quasi_2 = sum(belief_vector_pred)
            for i in range(len(belief_vector_pred)):
                belief_vector_pred[i]/=quasi_2 #normalization for belief of predator

        else:            
            steps += 1
            #if we arent certain of pred loc, survey node with highest probability
            if possible_pred_pos == predator_8.pos:
            #if we find pred, survey same node for prey
                for i in range(len(belief_vector_pred)):
                    if i == possible_pred_pos-1:
                        belief_vector_pred[i] = 1
                    else:
                        belief_vector_pred[i] = 0

                if possible_pred_pos == prey_8.pos:
                    for i in range(len(belief_vector_prey)):
                        if i == possible_pred_pos-1:
                            belief_vector_prey[i] = 1
                        else:
                            belief_vector_prey[i] = 0
                else:
                    prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of prey not being in that position
                    belief_vector_prey[possible_pred_pos-1] = 0
                    for i in range(len(belief_vector_prey)):
                            belief_vector_prey[i] = belief_vector_prey[i]/prey_not

            else:
                #if we dont find pred, update belief vector for pred
                pred_not = 1 - belief_vector_pred[possible_pred_pos-1]
                belief_vector_pred[possible_pred_pos-1] = 0
                for i in range(len(belief_vector_pred)):
                        belief_vector_pred[i] = belief_vector_pred[i]/pred_not

                if possible_pred_pos == prey_8.pos:
                    for i in range(len(belief_vector_prey)):
                        if i == possible_pred_pos-1:
                            belief_vector_prey[i] = 1
                        else:
                            belief_vector_prey[i] = 0
                else:
                    prey_not = 1-belief_vector_prey[possible_pred_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of prey not being in that position
                    belief_vector_prey[possible_pred_pos-1] = 0
                    for i in range(len(belief_vector_prey)):
                            belief_vector_prey[i] = belief_vector_prey[i]/prey_not

            quasi_1 = sum(belief_vector_pred)
            for i in range(len(belief_vector_pred)):
                belief_vector_pred[i]/=quasi_1 #normalization for predator belief, to ensure sum doesnt exceed 1


        path.append(agent_pos)
        #move prey
        prey_8.prey_move(g)

        if (prey_8.pos == agent_pos): #if prey luckily moves to location of agent
            break
        
        #move predator
        predator_8.pred_distracted_move(g, agent_pos)

        if(predator_8.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
        #belief update for both prey and predator for next timestep
        temp_belief_vector_pred = belief_vector_pred.copy()
        for i in range(len(belief_vector_pred)):
            mag_sum=0 #no chance of predator staying at same node in next timestep
            i_neighbours = g.graph[i+1] #getting neighbours
            for bloke in i_neighbours: 
                bloke_neighbours = g.graph[bloke] #finding neighbours of neighbours one of which will be Node I
                worthy_blokes=[]
                shortest_distance = 50
                for b_n in bloke_neighbours: #finding shortest paths from each neighbour and appending shortest paths to list
                    worthy_bloke_path = BFS(b_n, agent_pos, g.graph)
                    #saving shortest path(s)
                    if(len(worthy_bloke_path) <= shortest_distance):
                        if(len(worthy_bloke_path) == shortest_distance):
                            worthy_blokes.append(b_n)
                        else:
                            worthy_blokes = []
                            worthy_blokes.append(b_n)
                            shortest_distance = len(worthy_bloke_path)
                mag_sum = mag_sum + ((temp_belief_vector_pred[bloke-1])*(0.4/len(bloke_neighbours))) #the node could be chosen on a random basis while distracted
                mag_sum = mag_sum + (0.6*(temp_belief_vector_pred[bloke-1])*worthy_blokes.count(i+1)/len(worthy_blokes)) #if the node i is the next node on one of the shortest paths to agent from neighbour, calculate 0.6 probability on the basis of number of shortest paths from that neighbour
                """neighbour_path = BFS(bloke, agent_pos, g.graph)  #finding shortest path to agent position
                #print(neighbour_path)
                if len(neighbour_path) > 1:
                    mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/len(g.graph[bloke])))
                    if (neighbour_path[1] == i+1): #if the next position after start in the shortest path is the current position
                        mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.6))
                #else:
                    #mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/(len(g.graph[bloke])-1)))"""
            belief_vector_pred[i] = mag_sum #assigning probability

        quasi_3 = sum(belief_vector_pred)
        for i in range(len(belief_vector_pred)):
            belief_vector_pred[i]/=quasi_3

        temp_belief_vector_prey = belief_vector_prey.copy() #making a temporary belief vector for correct calculations
        for i in range(len(belief_vector_prey)):
            mag_sum = temp_belief_vector_prey[i]/(len(g.graph[i+1])+1)#prob of prey staying in same position
            i_neighbours = g.graph[i+1] #getting neighbours of current position
            #print(g.graph)
            #print(i+1)
            #print(i_neighbours)
            for bloke in i_neighbours:
                mag_sum = mag_sum + (temp_belief_vector_prey[bloke-1]/(len(g.graph[bloke])+1))#prob of prey moving to from neighbour bloke to node i
            belief_vector_prey[i] = mag_sum #assigning probability


    if(steps > 1000):
        fail = 2     
    return fail, path

In [12]:
wins = 0
losses = 0
timeout = 0
times = 0
win_dist = []
loss_dist = []

while times < 3000:
    g = Graph()
    graph = g.addEdge()
    while graph:
        graph = g.addEdge()
    res, path = Agent8(g)
    
    if(res == 0):
        wins += 1
        win_dist.append(len(path))
    elif(res == 1):
        losses += 1
        loss_dist.append(len(path))
    else:
        timeout += 1
    #print("Simulation Number " + str(times+1) + " completed.")
    times += 1
    
print('Number of wins: ' + str(wins))
print("Max Steps Taken During a Win: " + str(max(win_dist)-1))
print("Average Steps Taken During a Win: " + str((sum(win_dist)/wins)-1))
print('Number of losses: ' + str(losses))
print("Max Steps Taken During a Loss: " + str(max(loss_dist)-1))
print("Average Steps Taken During a Loss: " + str((sum(loss_dist)/losses)-1))
print('Number of timeouts: ' + str(timeout))
print("Success Rate: " + str((wins/times)))

Number of wins: 1528
Max Steps Taken During a Win: 141
Average Steps Taken During a Win: 23.160340314136125
Number of losses: 1472
Max Steps Taken During a Loss: 125
Average Steps Taken During a Loss: 20.485054347826086
Number of timeouts: 0
Success Rate: 0.5093333333333333
